In [8]:
import json
import re
import ast

In [ ]:
def parse_llm_response(response):
    """
    Robustly parse JSON or Python-like structures from an LLM response.
    Returns the loaded object (dict or list), or None if parsing fails.
    """
    if not response or not isinstance(response, str):
        print('response is empty or not a string')
        print(type(response))
        return None

    # Remove markdown code fences
    response_clean = re.sub(r'```(?:json)?', '', response).strip()

    # Extract JSON or Python literal between the first { } or [ ]
    match = re.search(r'(\{.*\}|\[.*\])', response_clean, re.DOTALL)
    if not match:
        print('match is None')
        return None

    payload = match.group(1)

    # Remove trailing commas before } or ]
    payload = re.sub(r',\s*([\}\]])', r'\1', payload)
    try:
        return json.loads(payload)
    except json.JSONDecodeError:
        # Fallback to Python literal eval
        try:
            return ast.literal_eval(payload)
        except Exception:
            print('ast.literal_eval failed')
            return None

In [37]:
response ="""{
    "understanding": "Change all text in the PowerPoint to gray color across all slides.",
    "tasks": [
        {
            "page number": 1,
            "description": "Change all text in slide 1 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 1",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 2,
            "description": "Change all text in slide 2 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 2",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 3,
            "description": "Change all text in slide 3 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 3",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 4,
            "description": "Change all text in slide 4 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 4",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 5,
            "description": "Change all text in slide 5 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 5",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 6,
            "description": "Change all text in slide 6 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 6",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 7,
            "description": "Change all text in slide 7 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 7",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 8,
            "description": "Change all text in slide 8 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 8",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 9,
            "description": "Change all text in slide 9 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 9",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 10,
            "description": "Change all text in slide 10 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 10",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        }
    ]
}"""

In [38]:
parse_llm_response(response)

{'understanding': 'Change all text in the PowerPoint to gray color across all slides.',
 'tasks': [{'page number': 1,
   'description': 'Change all text in slide 1 to gray',
   'target': 'All text elements (title, body, shapes, text boxes, etc.) on slide 1',
   'action': 'Change font color',
   'contents': {'color': '#808080'}},
  {'page number': 2,
   'description': 'Change all text in slide 2 to gray',
   'target': 'All text elements (title, body, shapes, text boxes, etc.) on slide 2',
   'action': 'Change font color',
   'contents': {'color': '#808080'}},
  {'page number': 3,
   'description': 'Change all text in slide 3 to gray',
   'target': 'All text elements (title, body, shapes, text boxes, etc.) on slide 3',
   'action': 'Change font color',
   'contents': {'color': '#808080'}},
  {'page number': 4,
   'description': 'Change all text in slide 4 to gray',
   'target': 'All text elements (title, body, shapes, text boxes, etc.) on slide 4',
   'action': 'Change font color',
   'c

In [21]:
import win32com.client
import pywintypes
import openai
from openai import OpenAI
import re

import json
import re
import ast

import re
import json
import ast
def get_simple_powerpoint_info():
    """
    현재 열려있는 PowerPoint의 페이지 수와 파일 이름만 가져옵니다.
    """
    try:
        # PowerPoint 애플리케이션에 연결
        ppt_app = win32com.client.GetObject(Class="PowerPoint.Application")
        
        # PowerPoint가 실행 중이고 열린 프레젠테이션이 있는지 확인
        if not ppt_app or not hasattr(ppt_app, 'ActivePresentation'):
            return "PowerPoint가 실행 중이 아니거나 열린 프레젠테이션이 없습니다."
        
        # 활성 프레젠테이션 가져오기
        presentation = ppt_app.ActivePresentation
        
        # 파일 이름과 페이지 수 가져오기
        file_name = presentation.Name
        slide_count = presentation.Slides.Count
        
        return {
            "파일 이름": file_name,
            "슬라이드 수": slide_count
        }
        
    except Exception as e:
        return f"오류 발생: {str(e)}"


In [24]:
get_simple_powerpoint_info()

{'파일 이름': 'math_simple.pptx', '슬라이드 수': 10}

In [10]:

def create_agent_user_prompt(page_number):
    prompt = f"""
Slide information:
- Page number: {page_number}
"""
    return prompt

In [12]:
import win32com.client
import os
import shutil

# PowerPoint 애플리케이션 시작
ppt = win32com.client.Dispatch("PowerPoint.Application")
ppt.Visible = True  # PowerPoint 창 표시

# 기존 파일 열기 또는 새 프레젠테이션 생성
# presentation = ppt.Presentations.Open(r"C:\path\to\your\presentation.pptx")
# 또는 새 프레젠테이션: 
# presentation = ppt.Presentations.Add()
# 첫 번째 슬라이드 가져오기

ppt = win32com.client.GetObject(Class="PowerPoint.Application")
presentation = ppt.ActivePresentation
slide = presentation.Slides(1)

# 예시: 텍스트 수정
shape = slide.Shapes(1)
original_text = shape.TextFrame.TextRange.Text
print(f"원본 텍스트: {original_text}")

# 텍스트 변경
shape.TextFrame.TextRange.Text = "수정된 텍스트"
print(f"변경된 텍스트: {shape.TextFrame.TextRange.Text}")

file_path = r"./presentation.pptx"
presentation.Save()
backup_path = file_path.replace('.pptx', '_backup.pptx')

# 백업 생성
# shutil.copy2(file_path, backup_path)

# # presentation = ppt.Presentations.Open(file_path)
# # slide = presentation.Slides(1)
# # shape = slide.Shapes(1)

# # # 수정
# # shape.TextFrame.TextRange.Text = "수정된 텍스트"
# # presentation.Save()

# # 복원이 필요한 경우
# presentation.Close()
# os.remove(file_path)
# shutil.move(backup_path, file_path)
presentation = ppt.Presentations.Open(file_path)

원본 텍스트: 수정된 텍스트
변경된 텍스트: 수정된 텍스트


com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024894), None)

In [14]:
import win32com.client as win32
import time
import os
from datetime import datetime

ppt = win32.Dispatch("PowerPoint.Application")
ppt.Visible = True

src_path = r"C:\Users\ben81\Documents\EditPPT\sample_ppt\test.pptx"
backup_dir = r"C:\Users\ben81\Documents\EditPPT\sample_ppt\backup"

presentation = ppt.Presentations.Open(src_path)

def backup_with_timestamp(pres, backup_dir):
    os.makedirs(backup_dir, exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_path = os.path.join(backup_dir, f"backup_{ts}.pptx")
    pres.SaveCopyAs(backup_path)
    print(f"백업 저장: {backup_path}")

# 수정 1
slide = presentation.Slides(1)
shape = slide.Shapes.AddTextbox(1, 100, 100, 400, 100)
shape.TextFrame.TextRange.Text = "수정 1"
backup_with_timestamp(presentation, backup_dir)

time.sleep(1)

# 수정 2
shape.TextFrame.TextRange.Text = "수정 2"
backup_with_timestamp(presentation, backup_dir)


백업 저장: C:\Users\ben81\Documents\EditPPT\sample_ppt\backup\backup_20260119_203444.pptx
백업 저장: C:\Users\ben81\Documents\EditPPT\sample_ppt\backup\backup_20260119_203445.pptx


In [ ]:
import win32com.client as win32
import os

ppt = win32.Dispatch("PowerPoint.Application")
ppt.Visible = True

presentation = ppt.ActivePresentation

original_path = r"C:\Users\ben81\Documents\EditPPT\sample_ppt\test.pptx"
backup_path = r"C:\Users\ben81\Documents\EditPPT\sample_ppt\backup\backup_20260119_203446.pptx"

# 1) 백업본 생성 (원본은 그대로 두고 복사 저장)
presentation.SaveCopyAs(backup_path)

# 2) 원본 닫기 (필요시 Save 호출 여부 선택)
presentation.Close()

# 3) 방금 만든 backup.pptx 열기
backup_presentation = ppt.Presentations.Open(backup_path)